In [1]:
# -*- coding: utf-8 -*-
import json
import cv2 as cv
# import cv2_imshow

# part 1:

def load_obj_each_frame(data_file):
  with open(data_file, 'r') as file:
    frame_dict = json.load(file)
  return frame_dict
def draw_target_object_center(video_file,obj_centers):
  count = 0
  cap = cv.VideoCapture(video_file)
  frames = []
  ok, image = cap.read()
  vidwrite = cv.VideoWriter("part_1_demo.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700,500))
  while ok:
    pos_x,pos_y = obj_centers[count]
    count+=1
    ######!!!!#######
    image = cv.resize(image, (700, 500)) # make sure your video is resize to this size, otherwise the coords in the data file won't work !!!
    ######!!!!#######
    image = cv.circle(image, (int(pos_x),int(pos_y)), 1, (0,0,255), 2)
    vidwrite.write(image)
    ok, image = cap.read()
  vidwrite.release()


frame_dict = load_obj_each_frame("object_to_track.json")
video_file = "commonwealth.mp4"
draw_target_object_center(video_file,frame_dict['obj'])

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [2]:
import numpy as np
import json
import cv2 as cv

In [3]:
file_path = 'part_1_object_tracking.json'

with open(file_path, 'r') as file:
    object_pos = json.load(file)

In [4]:
print(object_pos)

{'obj': [[-1, -1], [312, 228], [311, 228], [318, 205], [-1, -1], [-1, -1], [307, 225], [306, 224], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [287, 207], [287, 206], [286, 205], [286, 203], [-1, -1], [285, 202], [284, 202], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [278, 198], [276, 197], [-1, -1], [-1, -1], [275, 196], [-1, -1], [272, 194], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [262, 186], [261, 186], [260, 184], [260, 185], [259, 184], [258, 184], [257, 183], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [252, 176], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, -1], [-1, 

In [5]:
def predict(x, P, A, Q):
    # Make sure that x is a 1D array with shape (2,)
    x = np.array(x).flatten()

    # Predict the next state
    x_pred = A.dot(x)

    # Predict the next covariance matrix
    P_pred = A.dot(P).dot(A.T) + Q

    return x_pred, P_pred


In [6]:
def update(x_pred, P_pred, z, H, R):
    # Ensure x_pred is a 2-element 1D array
    x_pred = np.array(x_pred).flatten()

    # Innovation or residual, should be a 2-element 1D array
    y = z - H.dot(x_pred)
    # System uncertainty, should be a 2x2 matrix
    S = H.dot(P_pred).dot(H.T) + R
    # Kalman gain, should be a 2x2 matrix
    K = P_pred.dot(H.T).dot(np.linalg.inv(S))
    # Updated state estimate, should be a 2-element 1D array
    x_update = x_pred + K.dot(y)
    I = np.eye(P_pred.shape[0])  # Identity matrix
    # Updated estimate uncertainty, should be a 2x2 matrix
    P_update = (I - K.dot(H)).dot(P_pred)

    return x_update, P_update


In [66]:
def draw_target_object_center(video_file, obj_centers, forward_pass = True):
    P = np.eye(2) 
    if(forward_pass):
        A = np.eye(2) * 0.9972
        x = [312, 228]
    else:
        A = np.eye(2) * 1.0028
        x = [155.48221359647187, 95.25131111414534]
    H = np.eye(2)
    Q = np.eye(2) * 0.01
    R = np.eye(2) * 1
    count = 0
    cap = cv.VideoCapture(video_file)
    
    
    positions = {}
    positions["obj"] = []
    
    frames = []  
    ok, image = cap.read()
    vidwrite = cv.VideoWriter("part_1_demo_with_kalman.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700,500))
    
    
    while ok:
        if(count == 0 and forward_pass):
            count += 1
            continue
        elif count == 249 and forward_pass:
            break
        # print(count)
        pos_x, pos_y = obj_centers[count]
        count += 1
        if pos_x != -1 and pos_y != -1:
            z = np.array([pos_x, pos_y])
            if count == 1:
                x = np.array([pos_x, pos_y])
                P = np.diag([1, 1])
            else:

                x_pred, P_pred = predict(x, P, A, Q) #predict next frame 

                x_update, P_update = update(x_pred, P_pred, z, H, R) #
                
                x_update = x_update.tolist()
                
                frames.append([pos_x, pos_y])
                positions["obj"] += [[pos_x, pos_y]]
                # print(x_update)
                image = cv.resize(image, (700, 500))
                # print(x_update[0], x_update[1])
                # if len(frames) > 1:
                #     cv.polylines(image, [np.array(frames)], False, (0, 0, 255), thickness=1)

                x, P = x_update, P_update
        else:
            # print(x)
            x, P = predict(x, P, A, Q)
            x = x.tolist() #check the rounding issue
            frames.append([x[0], x[1]])
            positions["obj"] += [[x[0], x[1]]]

            # cv.polylines(image, [np.array(frames)], False, (0, 0, 255), thickness=1)
        image = cv.resize(image, (700, 500))
        # image = cv.circle(image, (pos_x, pos_y), 1, (0, 0, 255), 2)

        vidwrite.write(image)
        ok, image = cap.read()
    vidwrite.release()
    if(forward_pass):
       filename = 'filled_positions.json'
    else:
        filename = 'backwards_filled_positions.json'
        frames = frames[::-1]
        positions["obj"] = positions["obj"][::-1]
    
    with open(filename, 'w') as f:
        json.dump(positions, f)




In [69]:
file_path = 'part_1_object_tracking.json'

with open(file_path, 'r') as file:
    object_pos = json.load(file)

draw_target_object_center(video_file, object_pos['obj'])


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [70]:
file_path = 'filled_positions.json'

with open(file_path, 'r') as file:
    object_pos = json.load(file)
back_pos = [[-1, -1]] + object_pos['obj']
draw_target_object_center(video_file, back_pos[::-1], False)


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [12]:
video_file = "commonwealth.mp4"
draw_target_object_center(video_file, object_pos['obj'])

OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


[312, 228]
[312, 228]
[312, 228]
[314, 222]
[314, 222]
[314, 222]
[313, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[312, 223]
[308, 220]
[305, 218]
[302, 216]
[300, 214]
[300, 214]
[298, 213]
[296, 212]
[296, 212]
[296, 212]
[296, 212]
[296, 212]
[296, 212]
[296, 212]
[294, 210]
[292, 209]
[292, 209]
[292, 209]
[290, 208]
[290, 208]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[288, 207]
[285, 205]
[283, 203]
[281, 201]
[279, 199]
[277, 198]
[275, 197]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[273, 196]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]
[271, 194]

In [62]:
def create_video_path(video_file, obj_centers):
    obj_centers = [[int(round(x[0])), int(round(x[1]))] for x in obj_centers]
    count = 0
    cap = cv.VideoCapture(video_file)
    ok, image = cap.read()
    vidwrite = cv.VideoWriter(
        "part_1_demo_with_kalman.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700, 500))

    while ok:
        if(count == 0):
            count += 1
            continue
        elif count == 248:
            break
        # print(count)
        pos_x, pos_y = obj_centers[count]
        count += 1
        image = cv.resize(image, (700, 500))
        # print([np.array(obj_centers[:count+1])])
        cv.polylines(image, [np.array(obj_centers[:count+1])],
                     False, (0, 0, 255), thickness=2)
        image = cv.circle(image, (pos_x, pos_y), 1, (0, 0, 255), 2)        
        vidwrite.write(image)
        ok, image = cap.read()
    vidwrite.release()



In [71]:
file_path = 'backwards_filled_positions.json'

with open(file_path, 'r') as file:
    object_pos = json.load(file)
    

create_video_path(video_file, object_pos['obj'])


OpenCV: FFMPEG: tag 0x5634504d/'MP4V' is not supported with codec id 12 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'


In [17]:
print(len(object_pos["obj"]))


248


In [ ]:

# part 2:

def draw_object(object_dict,image,color = (0, 255, 0), thickness = 2,c_color= \
                (255, 0, 0)):
  # draw box
  x = object_dict['x_min']
  y = object_dict['y_min']
  width = object_dict['width']
  height = object_dict['height']
  image = cv.rectangle(image, (x, y), (x + width, y + height), color, thickness)
  return image

def draw_objects_in_video(video_file,frame_dict):
  count = 0
  cap = cv.VideoCapture(video_file)
  frames = []
  ok, image = cap.read()
  vidwrite = cv.VideoWriter("part_2_demo.mp4", cv.VideoWriter_fourcc(*'MP4V'), 30, (700,500))
  while ok:
    ######!!!!#######
    image = cv.resize(image, (700, 500)) # make sure your video is resize to this size, otherwise the coords in the data file won't work !!!
    ######!!!!#######
    obj_list = frame_dict[str(count)]
    for obj in obj_list:
      image = draw_object(obj,image)
    vidwrite.write(image)
    count+=1
    ok, image = cap.read()
  vidwrite.release()

frame_dict = load_obj_each_frame("frame_dict.json")
video_file = "commonwealth.mp4"
draw_objects_in_video(video_file,frame_dict)